In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport

from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor

from sklearn.metrics import explained_variance_score,max_error, mean_absolute_error,mean_squared_error, r2_score

from pycaret.regression import *

In [3]:
df = pd.read_csv(r"/mnt/c/Users/User Ambev/Desktop/gacathon-IfLittleBackNeck-2020/Datacase1_PartA_Train.csv", 
                 sep=';', index_col=0)

In [6]:
def feature_eng(data):
    random_vars = ['Packed_Volume',
       'Beer_Brewed_HG', 'Racked_Volume', 'Filter_Total_volume',
       'Line_1_Packed_Volume', 'Line_2_Packed_Volume', 'Line_3_Packed_Volume',
       'Line_4_Packed_Volume', 'Line_5_Packed_Volume', 'Line_6_Packed_Volume']
    dependent_var = 'Main_Incoming_Water'
    
    combinator = PolynomialFeatures(degree = 2,interaction_only = True)
    combinned_features = combinator.fit_transform(data[random_vars].fillna(data[random_vars].median()))
    new_random_feature_names = combinator.get_feature_names(random_vars)
    new_random_feature_names = [i.replace(' ',' X ') for i in new_random_feature_names]
    
    new_features_df = pd.DataFrame(combinned_features,columns = new_random_feature_names)
    new_features_df[dependent_var] = data[dependent_var].values
    new_features_df['log_target'] = np.log1p(new_features_df[dependent_var])
    new_features_df = new_features_df.drop(columns = ['1'])
    
    new_random_feature_names = [i for i in new_random_feature_names if i!='1']    
    
    return new_features_df

In [24]:
def feature_eng_test(data):
    random_vars = ['Packed_Volume',
       'Beer_Brewed_HG', 'Racked_Volume', 'Filter_Total_volume',
       'Line_1_Packed_Volume', 'Line_2_Packed_Volume', 'Line_3_Packed_Volume',
       'Line_4_Packed_Volume', 'Line_5_Packed_Volume', 'Line_6_Packed_Volume']
    
    combinator = PolynomialFeatures(degree = 2,interaction_only = True)
    combinned_features = combinator.fit_transform(data[random_vars].fillna(data[random_vars].median()))
    new_random_feature_names = combinator.get_feature_names(random_vars)
    new_random_feature_names = [i.replace(' ',' X ') for i in new_random_feature_names]
    
    new_features_df = pd.DataFrame(combinned_features,columns = new_random_feature_names)
    new_features_df['Observation_Id'] = data['Observation_Id']
    new_features_df = new_features_df.drop(columns = ['1'])
    
    new_random_feature_names = [i for i in new_random_feature_names if i!='1']    
    
    return new_features_df

In [7]:
df_features = feature_eng(df)

In [35]:
df['log_target'] = np.log1p(df['Main_Incoming_Water'])

In [36]:
exp_reg = setup(df.drop('Main_Incoming_Water',axis=1), target = 'log_target')

 
Setup Succesfully Completed.


,Description,Value
0,session_id,6803
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(2500, 11)"
4,Missing Values,True
5,Numeric Features,10
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [37]:
# cat = create_model('catboost')
et = create_model('et')
# xgb = create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0764,0.0301,0.1734,0.8916,0.0574,0.0466
1,0.0778,0.0297,0.1725,0.9091,0.0526,0.0410
2,0.0503,0.0095,0.0977,0.9692,0.0328,0.0300
3,0.0797,0.0283,0.1681,0.9363,0.0531,0.0411
4,0.0672,0.0225,0.1499,0.9349,0.0543,0.0432
5,0.0611,0.0237,0.1538,0.9193,0.0486,0.0339
6,0.0564,0.0112,0.1061,0.9572,0.0378,0.0341
7,0.0640,0.0185,0.1361,0.9337,0.0453,0.0359
8,0.0660,0.0370,0.1923,0.8554,0.0594,0.0373
9,0.0560,0.0134,0.1160,0.9455,0.0449,0.0395


In [38]:
tuned_et = tune_model(et)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.0851,0.0323,0.1797,0.8837,0.0598,0.0529
1,0.0895,0.0368,0.1917,0.8877,0.0593,0.0476
2,0.0558,0.0107,0.1032,0.9656,0.0340,0.0332
3,0.0984,0.0368,0.1919,0.9170,0.0612,0.0516
4,0.0779,0.0234,0.1529,0.9323,0.0553,0.0496
5,0.0696,0.0279,0.1671,0.9047,0.0532,0.0393
6,0.0658,0.0128,0.1132,0.9512,0.0412,0.0402
7,0.0817,0.0256,0.1601,0.9082,0.0524,0.0457
8,0.0744,0.0384,0.1959,0.8499,0.0629,0.0458
9,0.0680,0.0161,0.1268,0.9349,0.0521,0.0511


In [39]:
test_df = pd.read_csv(r"/mnt/c/Users/User Ambev/Desktop/gacathon-IfLittleBackNeck-2020/Datacase1_PartA_Test.csv", 
                      sep=';')

In [40]:
preds = predict_model(tuned_et, data=test_df)[['Observation_Id', 'Label']]

In [41]:
preds['Label'] = np.expm1(preds['Label'])

In [42]:
preds.to_csv(r"/mnt/c/Users/User Ambev/Desktop/predictions_trial3.csv")